<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/SFS/getData/SFS_scout_runs_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes:
- (Checked with @Phil + @Neil) as to which of the runs is the _latest_ one to look at from [aws bucket](https://noaa-reanalyses-pds.s3.amazonaws.com/index.html#analyses/scout_runs/)
- @Phil confirms: `3dvar_coupledreanl_scoutrun_v1.01 is the folder that is being updated.`

# Set Inputs

In [ ]:
url_base = 'https://noaa-reanalyses-pds.s3.amazonaws.com/analyses/scout_runs/3dvar_coupledreanl_scoutrun_v1.01/'

out_path = 'drive/MyDrive/UFS-no-RTOFS/SFS/datasets/scout_runs/'

In [ ]:
# As of 12/02/2025
# Stream-1
# starts on 1979/01/01 12UTC and available up to 1979/04/12 06UTC
sd, ed = '1979-01-02', '1979-04-01'

# Stream-2
# starts on 2023/01/01 12UTC and available up to 2023/04/04 12UTC
# sd, ed = '2023-01-02', '2023-04-01'

ana_freq = '6h'
res_offSet = 3

In [ ]:
import pandas as pd
import xarray as xr

In [ ]:
for date in pd.date_range(start=sd, end=ed, freq=ana_freq):
  restart_date = date + pd.Timedelta(hours=res_offSet)

  url_suff = f'{date.strftime('%Y%m%d%H')}/gdas.{date.strftime('%Y%m%d')}/{date.strftime('%H')}/model/ocean/restart/'
  resName = f'{restart_date.strftime("%Y%m%d.%H%M%S")}.MOM.res.nc'

  url = url_base + url_suff + resName

  print(f'Fetching: [{resName}]')
  print('')

  !wget -q $url

  # Fix time in restart file
  ds_in = xr.open_dataset(f'{resName}', decode_times=False)
  ds_in.coords['Time'] = [restart_date.to_datetime64()]
  ds_in.to_netcdf(f'{out_path}{resName}')

  !rm $resName